In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import h5py
import warnings
warnings.filterwarnings('ignore')

In [2]:
file_electron = "SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5"
file_photon = "SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5"

with h5py.File(file_electron, "r") as f1:
    X_elec = np.array(f1['X'][:])
    y_elec = np.array(f1['y'][:])
with h5py.File(file_photon, "r") as f2:
    X_phot = np.array(f2['X'][:])
    y_phot = np.array(f2['y'][:])
print(X_elec.shape)
print(X_phot.shape)

(249000, 32, 32, 2)
(249000, 32, 32, 2)


In [3]:
num_classes = 2
input_shape = (32, 32, 2)
X = np.append(X_elec, X_phot, axis=0)
y = np.append(y_elec, y_phot)
X.shape

(498000, 32, 32, 2)

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=4, stratify=y)

In [5]:
y_train = keras.utils.to_categorical(y_train, num_classes=2)
y_test = keras.utils.to_categorical(y_test, num_classes=2)

In [6]:
learning_rate = 0.0001
batch_size = 256
num_epochs = 50
patch_size = 2
num_patches = (input_shape[0]//patch_size)**2
projection_dim = 64
num_heads = 2
transformer_units = [
    projection_dim * 2,
    projection_dim,
]
transformer_layers = 2
mlp_head_units = [512, 256]

In [7]:
def mlp(x, hidden_units):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
    return x

In [8]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [9]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [10]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    patches = Patches(patch_size)(inputs)
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    for _ in range(transformer_layers):
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0
        )(x1, x1)
        x2 = layers.Add()([attention_output, encoded_patches])
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = mlp(x3, hidden_units=transformer_units)
        encoded_patches = layers.Add()([x3, x2])
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    features = mlp(representation, hidden_units=mlp_head_units)
    outputs = layers.Dense(num_classes, activation='softmax')(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [11]:
def run_experiment(model):
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=[tf.keras.metrics.AUC()])
    
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_auc', factor=0.2,
                              patience=10, min_lr=1e-10   , verbose=1)
    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[reduce_lr]
    )
    return model, history

In [12]:
vit_classifier = create_vit_classifier()
with tf.device('/gpu:0'):
    model, history = run_experiment(vit_classifier)

2022-03-23 01:37:27.243773: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 01:37:27.261132: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 01:37:27.263336: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 01:37:27.266516: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/50
1401/1401 [==============================] - 91s 62ms/step - loss: 0.6781 - auc: 0.6304 - val_loss: 0.6397 - val_auc: 0.6838 - lr: 1.0000e-04
Epoch 2/50
1401/1401 [==============================] - 87s 62ms/step - loss: 0.6399 - auc: 0.6851 - val_loss: 0.6214 - val_auc: 0.7124 - lr: 1.0000e-04
Epoch 3/50
1401/1401 [==============================] - 86s 62ms/step - loss: 0.6218 - auc: 0.7124 - val_loss: 0.6009 - val_auc: 0.7418 - lr: 1.0000e-04
Epoch 4/50
1401/1401 [==============================] - 87s 62ms/step - loss: 0.6093 - auc: 0.7294 - val_loss: 0.6072 - val_auc: 0.7382 - lr: 1.0000e-04
Epoch 5/50
1401/1401 [==============================] - 87s 62ms/step - loss: 0.6012 - auc: 0.7394 - val_loss: 0.5904 - val_auc: 0.7528 - lr: 1.0000e-04
Epoch 6/50
1401/1401 [==============================] - 87s 62ms/step - loss: 0.5950 - auc: 0.7467 - val_loss: 0.5902 - val_auc: 0.7538 - lr: 1.0000e-04
Epoch 7/50
1401/1401 [==============================] - 87s 62ms/step - loss: 0.58

In [13]:
model.evaluate(x_test,y_test)

3113/3113 [==============================] - 42s 14ms/step - loss: 0.5674 - auc: 0.7775


[0.5674085021018982, 0.7774555683135986]

#### Results

Train AUC Score: 0.806

Validation AUC Score: 0.782

Test AUC Score: 0.777